# Low-Rank Matrix Completion (LRMC)

In [ ]:
import numpy as np

def Hankel_indexing(mat, tau1, tau2, k1, k2):
    N, T = mat.shape
    return mat[k1 : N - tau1 + 1 - k1, k2 : T - tau2 + 1 - k2]

def CP_reconstruct(Q, S, U, V):
    dim1 = Q.shape[0]
    tau1 = S.shape[0]
    dim2 = U.shape[0]
    tau2 = V.shape[0]
    N = dim1 + tau1 - 1
    T = dim2 + tau2 - 1
    mat = np.zeros((N, T))
    bin = np.zeros((N, T))
    for k1 in range(tau1):
        for k2 in range(tau2):
            temp = np.zeros((N, T))
            temp1 = Q @ np.diag(S[k1, :] * V[k2, :]) @ U.T
            temp[k1 : dim1 + k1, k2 : dim2 + k2] = temp1
            temp2 = np.zeros((N, T))
            temp2[k1 : dim1 + k1, k2 : dim2 + k2] = 1
            mat += temp
            bin += temp2
    return mat / bin

def update_cg(var, r, d, Ad, rold):
    alpha = rold / np.inner(d, Ad)
    var = var + alpha * d
    r = r - alpha * Ad
    rnew = np.inner(r, r)
    d = r + (rnew / rold) * d
    return var, r, d, rnew

def ell_q(sparse_mat, Q, S, U, V, tau1, tau2, rho):
    N, T = sparse_mat.shape
    temp = rho * Q
    for k1 in range(tau1):
        for k2 in range(tau2):
            ind = sparse_mat[k1 : N - tau1 + 1 + k1, 
                             k2 : T - tau2 + 1 + k2] != 0
            W_k1k2 = np.diag(S[k1, :] * V[k2, :]) @ U.T
            temp += ((Q @ W_k1k2) * ind) @ W_k1k2.T
    return temp

def conj_grad_q(sparse_mat, Q, S, U, V, tau1, tau2, rho, maxiter = 5):
    N, T = sparse_mat.shape
    R = Q.shape[1]
    q = np.reshape(Q, -1, order = 'F')
    temp1 = np.zeros((N - tau1 + 1, R))
    for k1 in range(tau1):
        for k2 in range(tau2):
            W_k1k2 = np.diag(S[k1, :] * V[k2, :]) @ U.T
            temp1 += sparse_mat[k1 : N - tau1 + 1 + k1, 
                                k2 : T - tau2 + 1 + k2] @ W_k1k2.T
    r = np.reshape(temp1 - ell_q(sparse_mat, Q, S, U, V, tau1, tau2, rho), 
                   -1, order = 'F')
    d = r.copy()
    rold = np.inner(r, r)
    for it in range(maxiter):
        D = np.reshape(d, (N - tau1 + 1, R), order = 'F')
        Ad = np.reshape(ell_q(sparse_mat, D, S, U, V, tau1, tau2, rho), 
                        -1, order = 'F')
        q, r, d, rold = update_cg(q, r, d, Ad, rold)
    return np.reshape(q, (N - tau1 + 1, R), order = 'F')

def ell_s(sparse_mat, Q, s_k1, k1, U, V, tau1, tau2, rho):
    N, T = sparse_mat.shape
    temp = rho * s_k1
    for k2 in range(tau2):
        ind = sparse_mat[k1 : N - tau1 + 1 + k1, 
                         k2 : T - tau2 + 1 + k2] != 0
        temp += np.diag(Q.T @ ((Q @ np.diag(s_k1 * V[k2, :]) 
                                @ U.T) * ind) @ U) * V[k2, :]
    return temp

def conj_grad_s(sparse_mat, Q, s_k1, k1, U, V, tau1, tau2, rho, maxiter = 5):
    N, T = sparse_mat.shape
    R = s_k1.shape[0]
    temp1 = np.zeros(R)
    for k2 in range(tau2):
        temp1 += np.diag(Q.T @ sparse_mat[k1 : N - tau1 + 1 + k1,
                                          k2 : T - tau2 + 1 + k2] @ U) * V[k2, :]
    r = temp1 - ell_s(sparse_mat, Q, s_k1, k1, U, V, tau1, tau2, rho)
    d = r.copy()
    rold = np.inner(r, r)
    for it in range(maxiter):
        Ad = ell_s(sparse_mat, Q, d, k1, U, V, tau1, tau2, rho)
        s_k1, r, d, rold = update_cg(s_k1, r, d, Ad, rold)
    return s_k1

def ell_u(sparse_mat, Q, S, U, V, tau1, tau2, rho):
    N, T = sparse_mat.shape
    temp = rho * U
    for k1 in range(tau1):
        for k2 in range(tau2):
            ind = sparse_mat[k1 : N - tau1 + 1 + k1, 
                             k2 : T - tau2 + 1 + k2] != 0
            H_k1k2 = Q @ np.diag(S[k1, :] * V[k2, :])
            temp += ((H_k1k2 @ U.T) * ind).T @ H_k1k2
    return temp

def conj_grad_u(sparse_mat, Q, S, U, V, tau1, tau2, rho, maxiter = 5):
    N, T = sparse_mat.shape
    R = U.shape[1]
    u = np.reshape(U, -1, order = 'F')
    temp1 = np.zeros((T - tau2 + 1, R))
    for k1 in range(tau1):
        for k2 in range(tau2):
            H_k1k2 = Q @ np.diag(S[k1, :] * V[k2, :])
            temp1 += sparse_mat[k1 : N - tau1 + 1 + k1, 
                               k2 : T - tau2 + 1 + k2].T @ H_k1k2
    r = np.reshape(temp1 - ell_u(sparse_mat, Q, S, U, V, tau1, tau2, rho), 
                   -1, order = 'F')
    d = r.copy()
    rold = np.inner(r, r)
    for it in range(maxiter):
        D = np.reshape(d, (T - tau2 + 1, R), order = 'F')
        Ad = np.reshape(ell_u(sparse_mat, Q, S, D, V, tau1, tau2, rho), 
                        -1, order = 'F')
        u, r, d, rold = update_cg(u, r, d, Ad, rold)
    return np.reshape(u, (T - tau2 + 1, R), order = 'F')

def ell_v(sparse_mat, Q, S, U, v_k2, k2, tau1, tau2, rho):
    N, T = sparse_mat.shape
    temp = rho * v_k2
    for k1 in range(tau1):
        ind = sparse_mat[k1 : N - tau1 + 1 + k1, 
                         k2 : T - tau2 + 1 + k2] != 0
        temp += np.diag(Q.T @ ((Q @ np.diag(S[k1, :] * v_k2) 
                                @ U.T) * ind) @ U) * S[k1, :]
    return temp

def conj_grad_v(sparse_mat, Q, S, U, v_k2, k2, tau1, tau2, rho, maxiter = 5):
    N, T = sparse_mat.shape
    R = v_k2.shape[0]
    temp1 = np.zeros(R)
    for k1 in range(tau1):
        temp1 += np.diag(Q.T @ sparse_mat[k1 : N - tau1 + 1 + k1,
                                          k2 : T - tau2 + 1 + k2] @ U) * S[k1, :]
    r = temp1 - ell_v(sparse_mat, Q, S, U, v_k2, k2, tau1, tau2, rho)
    d = r.copy()
    rold = np.inner(r, r)
    for it in range(maxiter):
        Ad = ell_v(sparse_mat, Q, S, U, d, k2, tau1, tau2, rho)
        v_k2, r, d, rold = update_cg(v_k2, r, d, Ad, rold)
    return v_k2

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def HTF_CP(dense_tensor, sparse_tensor, tau1, tau2, R, lmbda, rho, maxiter = 50):
    N, T, L = sparse_tensor.shape
    if np.isnan(sparse_tensor).any() == False:
        pos_test = np.where((dense_tensor != 0) & (sparse_tensor == 0))
    elif np.isnan(sparse_tensor).any() == True:
        pos_test = np.where((dense_tensor > 0) & (np.isnan(sparse_tensor)))
        sparse_tensor[np.isnan(sparse_tensor)] = 0
    tensor_hat = np.zeros((N, T, L))
    for t in range(L):
        Q = 0.01 * np.random.randn(N - tau1 + 1, R)
        S = np.ones((tau1, R))
        U = 0.01 * np.random.randn(T - tau2 + 1, R)
        V = np.ones((tau2, R))
        ind = sparse_tensor[:, :, t] != 0
        for it in range(maxiter):
            Q = conj_grad_q(sparse_tensor[:, :, t], Q, S, U, V, tau1, tau2, rho)
            for k1 in range(tau1):
                S[k1, :] = conj_grad_s(sparse_tensor[:, :, t], Q, S[k1, :], k1, U, V, 
                                       tau1, tau2, rho)
            U = conj_grad_u(sparse_tensor[:, :, t], Q, S, U, V, tau1, tau2, rho)
            for k2 in range(tau2):
                V[k2, :] = conj_grad_v(sparse_tensor[:, :, t], Q, S, U, V[k2, :], k2,
                                       tau1, tau2, rho)
            tensor_hat[:, :, t] = CP_reconstruct(Q, S, U, V)
    print('Iter: {}'.format(it + 1))
    print('MAPE: {:.6}'.format(compute_mape(dense_tensor[pos_test], 
                                            tensor_hat[pos_test])))
    print('RMSE: {:.6}'.format(compute_rmse(dense_tensor[pos_test], 
                                            tensor_hat[pos_test])))
    print()
    return tensor_hat

## HighD Dataset

Hyperparameters:

- On 30% trajectories: $\tau_1=3,\tau_2=4,R=10$;
- On 50% trajectories: $\tau_1=3,\tau_2=10,R=10$;
- On 70% trajectories: $\tau_1=4,\tau_2=15,R=10$.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import imageio as io
plt.rcParams['font.size'] = 12

def plot_speed_field(data, filename, x_scale=2, y_scale=3):
    fig = plt.figure(figsize = (2.5*2.5, 5))
    x = range(data.shape[1])
    x_new = [i * x_scale for i in x]
    y = range(data.shape[0])
    y_new = [i * y_scale for i in y]
    plt.matshow(data, cmap='jet_r', origin='lower', 
                extent=[x_new[0], x_new[-1], y_new[0], y_new[-1]],
                vmin = 5, vmax = 35, fignum = 1)
    plt.gca().xaxis.set_ticks_position('bottom')
    plt.xlabel('Time (s)')
    plt.ylabel('Location (m)')
    cbar = plt.colorbar(fraction = 0.015)
    cbar.ax.set_ylabel('Speed (m/s)')
    plt.show()
    fig.savefig(filename, bbox_inches = 'tight', dpi = 300)

dense_tensor = np.load('../datasets/HighD/speed_matrix_full_46.npy').transpose(1, 0, 2)
sparse_tensor = np.load('../datasets/HighD/speed_matrix_70_46.npy').transpose(1, 0, 2)

for lane in range(3):
    plot_speed_field(dense_tensor[:, :, lane],
                     'speed_matrix_full_46_lane{}.png'.format(lane + 1))
    plot_speed_field(sparse_tensor[:, :, lane],
                     'speed_matrix_70_46_lane{}.png'.format(lane + 1))

import time
start = time.time()
tau1 = 4
tau2 = 15
R = 10
lmbda = 0
rho = 1e+1
maxiter = 100
tensor_hat = HTF_CP(dense_tensor, sparse_tensor, tau1, tau2, R, lmbda, rho, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

for lane in range(3):
    plot_speed_field(tensor_hat[:, :, lane], 
                     'HighD_speed_field_70_HTF_CP_rec_lane{}.png'.format(lane + 1))

## CitySim Dataset

Hyperparameters:

- On 30% trajectories: $\tau_1=3,\tau_2=15,R=10$;
- On 50% trajectories: $\tau_1=3,\tau_2=15,R=10$;
- On 70% trajectories: $\tau_1=4,\tau_2=15,R=10$.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import imageio as io
plt.rcParams['font.size'] = 12

def plot_speed_field(data, filename, x_scale=2, y_scale=5):
    fig = plt.figure(figsize = (2.3*2.3, 1.8))
    x = range(data.shape[1])
    x_new = [i * x_scale for i in x]
    y = range(data.shape[0])
    y_new = [i * y_scale for i in y]
    plt.matshow(data, cmap='jet_r', origin='lower',
                extent=[x_new[0], x_new[-1], y_new[0], y_new[-1]],
                vmin = 14, vmax = 35, fignum = 1, aspect='auto')
    plt.gca().xaxis.set_ticks_position('bottom')
    plt.xlabel('Time (s)')
    plt.ylabel('Location (m)')
    cbar = plt.colorbar(fraction = 0.015)
    cbar.ax.set_ylabel('Speed (m/s)')
    plt.show()
    fig.savefig(filename, bbox_inches = 'tight', dpi = 300)

dense_tensor = np.load('../datasets/CitySim/speed_matrix_full.npy').transpose(1, 0, 2)
sparse_tensor = np.load('../datasets/CitySim/speed_matrix_70_FB.npy').transpose(1, 0, 2)

for lane in range(3):
    plot_speed_field(dense_tensor[:, :, lane],
                     'speed_matrix_full_lane{}.png'.format(lane + 1))
    plot_speed_field(sparse_tensor[:, :, lane],
                     'speed_matrix_70_lane{}.png'.format(lane + 1))

import time
start = time.time()
tau1 = 4
tau2 = 15
R = 10
lmbda = 0
rho = 1e+1
maxiter = 100
tensor_hat = HTF_CP(dense_tensor, sparse_tensor, tau1, tau2, R, lmbda, rho, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

for lane in range(3):
    plot_speed_field(tensor_hat[:, :, lane], 
                     'CitySim_speed_field_70_HTF_CP_rec_lane{}.png'.format(lane + 1))

### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>